In [10]:
BOATSPEEDS = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.4, 1.1, 3.2, 3.7, 2.8],
    [0, 0.3, 1.9, 3.7, 9.3, 13.0, 9.2],
    [0, 0.9, 3.7, 7.4, 14.8, 18.5, 13.0],
    [0, 1.3, 5.6, 9.3, 18.5, 24.1, 18.5],
]

WINDSPEEDS = [0, 9.3, 18.5, 27.8, 37.0]  # The row labels
ANGLES = [0, 20, 30, 45, 90, 135, 180]  # The column labels


def interpolate_boat_speed(true_windspeed, sailing_angle):
    sailing_angle = abs(sailing_angle)

    # Find the nearest windspeed values above and below the true windspeed
    lower_windspeed_index = max([i for i, ws in enumerate(WINDSPEEDS) if ws <= true_windspeed])
    upper_windspeed_index = (
        lower_windspeed_index + 1
        if lower_windspeed_index < len(WINDSPEEDS) - 1
        else lower_windspeed_index
    )

    # Find the nearest angle values above and below the sailing angle
    lower_angle_index = max([i for i, ang in enumerate(ANGLES) if ang <= sailing_angle])
    upper_angle_index = (
        lower_angle_index + 1 if lower_angle_index < len(ANGLES) - 1 else lower_angle_index
    )

    # Find the maximum angle and maximum windspeed based on the actual data in the table
    max_angle = max(ANGLES)
    max_windspeed = max(WINDSPEEDS)

    # Handle the case of maximum angle (use the dynamic max_angle)
    if upper_angle_index == len(ANGLES) - 1:
        lower_angle_index = ANGLES.index(max_angle) - 1
        upper_angle_index = ANGLES.index(max_angle)

    # Handle the case of the maximum windspeed (use the dynamic max_windspeed)
    if upper_windspeed_index == len(WINDSPEEDS) - 1:
        lower_windspeed_index = WINDSPEEDS.index(max_windspeed) - 1
        upper_windspeed_index = WINDSPEEDS.index(max_windspeed)

    # Perform linear interpolation
    lower_windspeed = WINDSPEEDS[lower_windspeed_index]
    upper_windspeed = WINDSPEEDS[upper_windspeed_index]
    lower_angle = ANGLES[lower_angle_index]
    upper_angle = ANGLES[upper_angle_index]

    boat_speed_lower = BOATSPEEDS[lower_windspeed_index][lower_angle_index]
    boat_speed_upper = BOATSPEEDS[upper_windspeed_index][lower_angle_index]

    interpolated_1 = boat_speed_lower + (true_windspeed - lower_windspeed) * (
        boat_speed_upper - boat_speed_lower
    ) / (upper_windspeed - lower_windspeed)

    boat_speed_lower = BOATSPEEDS[lower_windspeed_index][upper_angle_index]
    boat_speed_upper = BOATSPEEDS[upper_windspeed_index][upper_angle_index]

    interpolated_2 = boat_speed_lower + (true_windspeed - lower_windspeed) * (
        boat_speed_upper - boat_speed_lower
    ) / (upper_windspeed - lower_windspeed)

    interpolated_value = interpolated_1 + (sailing_angle - lower_angle) * (
        interpolated_2 - interpolated_1
    ) / (upper_angle - lower_angle)

    return interpolated_value

Boat Speed: 18.5 km/h


In [17]:
import pytest


def test_interpolate_boat_speed(true_windspeed, sailing_angle, answer):
    boat_speed = interpolate_boat_speed(true_windspeed, sailing_angle)
    return boat_speed == pytest.approx(answer)


# Corners of the table
assert test_interpolate_boat_speed(0, 0, 0)
assert test_interpolate_boat_speed(37.0, 180, 18.5)
assert test_interpolate_boat_speed(0, 180, 0)
assert test_interpolate_boat_speed(37.0, 0, 0)

# Windspeed is 0 and we have an angle
assert test_interpolate_boat_speed(0, 12, 0)
assert test_interpolate_boat_speed(0, 20, 0)
assert test_interpolate_boat_speed(0, 30, 0)
assert test_interpolate_boat_speed(0, 135, 0)
assert test_interpolate_boat_speed(0, 148.2, 0)

# Angle is 0 and we have a windspeed
assert test_interpolate_boat_speed(9.3, 0, 0)
assert test_interpolate_boat_speed(32.3, 0, 0)
assert test_interpolate_boat_speed(37.0, 0, 0)

# Other edge cases
assert test_interpolate_boat_speed(10.6, 180, 3.704347826)
assert test_interpolate_boat_speed(37.0, 35.0, 6.833333333)
assert test_interpolate_boat_speed(27.8, 102.7, 15.844222222)
assert test_interpolate_boat_speed(14.4, 30.0, 1.231521739)

# Both angle and windspeed have are directly in the table
assert test_interpolate_boat_speed(18.5, 45, 3.7)

# General case between angles
assert test_interpolate_boat_speed(12.0, 60, 2.905434783)
assert test_interpolate_boat_speed(5.3, 13.9, 0)